# Apply Quantization on my Llama-2 7B Fine-tuned Python Code Generator using GPTQ with AutoGPT and Huggingface

In my previous [notebook](https://github.com/edumunozsala/llama-2-7B-4bit-python-coder/blob/main/Llama-2-finetune-qlora-python-coder.ipynb), I showed you how to instruction fine-tune the new Llama 2 model, just released by Meta AI, to build a Python code generator in a few lines of code. This time we will describe how to quantize this model using the GPTQ quantization, now that it is integrated with the transformers.

Last week Hugging Face Hugging Face announced the compatibility of its transformers libraries with the AutoGPTQ library which allows us to quantize a large language model in 2, 3, or 4 bits using the GPTQ methodology

This notebook contains three sections:
- Load the original model, fine-tuned with QLoRa in 4-bits, and evaluate the performance when runing on a T4 instance in Colab.
- Quantize this original model to 4 bits using GPTQ with transformers and AUTOGPT and save it to Hugging Face Hub.
- Load the quantized model and evaluate the performance in a T4.

## Load the Llama-2 7B model in 4-bit without AutoGPTQ

### Installing the libraries

In [ ]:
! pip install "transformers==4.31.0" "bitsandbytes==0.40.2" "accelerate==0.21.0"

We load the model in 4 bits with bitsandbytes quantization

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the entire model on the GPU 0
device_map = {"": 0}

# If not defined
hf_model_repo='edumunozsala/llama-2-7b-int4-python-code-20k'

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)
# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, load_in_4bit=True, torch_dtype=torch.float16,
                                             device_map=device_map)


Lets check the memory usage

In [ ]:
!nvidia-smi

## Test Inference performance

Install the dataset library

In [ ]:
!pip install "datasets==2.13.0"

In [ ]:
from datasets import load_dataset
from random import randrange
import time

### Load the dataset with the instruction set

First, we load the dataset from huggingface. The dataset is [python_code_instructions_18k_alpaca](https://huggingface.co/datasets/iamtarun/python_code_instructions_18k_alpaca)

The dataset contains problem descriptions and code in python language. This dataset is taken from sahil2801/code_instructions_120k, which adds a prompt column in alpaca style.


In [ ]:
# The instruction dataset to use
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
# Load dataset from the hub
dataset = load_dataset(dataset_name, split="train[:10%]")
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
print(dataset[randrange(len(dataset))])


In [ ]:
# Check the dataset structure
print(dataset)

We would apply inference on those examples longer than 500 characters:

In [ ]:
# Sample the dataset
#sample_dataset= dataset[randrange(0,100,5)]
# Filter the dataset to get those which instruction is longer than 100 characters
responses_test = dataset.filter(lambda x: len(x['instruction'])>500)

print(len(responses_test))

Now, we run a for loop where we make inference and collect execution time and the result

In [ ]:
# Process the test examples
times = []
solutions = []

for example in responses_test:
  start = time.time()
  prompt = f"""### Instruction:
  Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

  ### Task:
  {example["instruction"]}

  ### Input:
  {example["input"]}

  ### Response:
  """
  # Tokenize the input
  input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  # Run the model to infere an output
  outputs = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.3)

  # Collect the output and the inference time
  end = time.time()
  print(start-end)
  times.append(start-end)
  solutions.append(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):])

print(times)

As a summary, we show the max, min and mean execution time

In [ ]:
import numpy as np

In [ ]:
print('Min time: ',np.min(times))
print('Max time: ',np.max(times))
print('Mean time: ',np.mean(times))

In average, **every inference takes about 17 - 19 seconds**.

## Quantize transformers model using auto-gptq, 🤗 transformers and optimum

The GPTQ algorithm requires to calibrate the quantized weights of the model by doing inference on the quantized model. The detailed quantization algorithm is described in [the original paper](https://arxiv.org/pdf/2210.17323.pdf).

For quantizing a model using auto-gptq, we need to pass a dataset to the quantizer. This can be achieved either by passing a supported default dataset among `['wikitext2','c4','c4-new','ptb','ptb-new']` or a list of strings that will be used as a dataset.

Installing the libraries

In [ ]:
!pip install -q -U transformers peft accelerate optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Hugging Face blog post recommends to install an explicitd version, but probably you can run the simple installation

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/
#!pip install auto-gptq

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.9 MB/s eta 0:00:00


In this example, we will try to quantize the model in 4-bit precision using the `"c4"` dataset and the tokenizer from our model. Supported precisions are `[2, 4, 6, 8]`. The other parameters takes the default value.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

# Set the model to load
hf_model_repo='edumunozsala/llama-2-7b-int4-python-code-20k'
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo, use_fast=True)
# Set quantization configuration
quantization_config = GPTQConfig(
     bits=4,
     group_size=128,
     dataset="c4",
     desc_act=False,
     tokenizer=tokenizer
)
# Load the model from HF
quant_model = AutoModelForCausalLM.from_pretrained(hf_model_repo, quantization_config=quantization_config, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

The parameters defined:

- **bits**: Precision of the quantization. You can set 4, 3, and 2.
- **dataset**: The dataset used for calibration. I would leave “c4“ which seems to yield reasonable results. Other datasets are supported according to the documentation.
- **desc_act**: Whether to quantize columns in order of decreasing activation size. Setting it to False can significantly speed up inference but the perplexity may become slightly worse. Also known as act-order.
- **tokenizer**: The tokenizer of Llama 2 7B that will be applied to c4.

Lets check the memory usage

In [ ]:
!nvidia-smi

Mon Sep  4 07:17:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    49W / 400W |  27249MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

You can make sure the model has been correctly quantized by checking the attributes of the linear layers, they should contain `qweight` and `qzeros` attributes that should be in `torch.int32` dtype.

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

Now, we run a simple example

In [ ]:
# Create an instruction
instruction="Write a Python function to display the first and last elements of a list."
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""
# Tokenize the input
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# Run the model to infere an output
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

# Print the result
print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")

## Share quantized models on 🤗 Hub

After quantizing the model, it can be used out-of-the-box for inference or you can push the quantized weights on the 🤗 Hub to share your quantized model with the community

In [ ]:
!pip install python-dotenv
!pip install huggingface_hub

You can log in to Hugging Face Hub interactively

In [ ]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()


Or you can provide .env file containing the Hugging Face token

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load the enviroment variables
load_dotenv()
# Login to the Hugging Face Hub
login(token=os.getenv("HF_HUB_TOKEN"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Now, we can upload our quantize model to the Hugging Face Hub

In [ ]:
quant_model.push_to_hub("edumunozsala/llama-2-7b-int4-GPTQ-python-code-20k")
tokenizer.push_to_hub("edumunozsala/llama-2-7b-int4-GPTQ-python-code-20k")

pytorch_model.bin:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edumunozsala/llama-2-7b-int4-GPTQ-python-code-20k/commit/ed458634c5162e5d755532fb10177864b3fa3b5c', commit_message='Upload tokenizer', commit_description='', oid='ed458634c5162e5d755532fb10177864b3fa3b5c', pr_url=None, pr_revision=None, pr_num=None)

## Loading the GPTQ Model from Hugging Face Hub and run an example

As usual, we import the libraries and load the tokenizer and quantized model

Installing the libraries

In [ ]:
!pip install -q -U transformers peft accelerate optimum

In [ ]:
#!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/
!pip install auto-gptq

Load the model, this time we do not need to especify any quantization mode. The model is already quantized, you can load models that have been quantized using the auto-gptq library out of the box from the 🤗 Hub directly using `from_pretrained` method.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "edumunozsala/llama-2-7b-int4-GPTQ-python-code-20k"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")


Once tokenizer and model has been loaded, let's generate some text. Before that, we can inspect the model to make sure it has loaded a quantized model, you can see the `QuantLinear()` layers.

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


Lets check the GPU VRAM

In [ ]:
!nvidia-smi

Mon Sep  4 17:30:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W /  70W |   5053MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Then we run an inference to test it

In [ ]:
instruction="Write a Python function to display the first and last elements of a list."
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=128, do_sample=True, top_p=0.9,temperature=0.3)

print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python function to display the first and last elements of a list.

### Input:


### Response:


Generated instruction:
def first_last(list):
    first = list[0]
    last = list[-1]
    return first, last



Everything looks fine.

## Test Inference performance

In [ ]:
!pip install "datasets==2.13.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4


In [ ]:
from datasets import load_dataset
from random import randrange
import time

### Load the dataset with the instruction set

In [ ]:
# The instruction dataset to use
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
# Load dataset from the hub
dataset = load_dataset(dataset_name, split="train[:10%]")
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
print(dataset[randrange(len(dataset))])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

dataset size: 1861
{'instruction': 'Write a Python script for recognizing facial features using OpenCV and Haar features.', 'input': '<Sample_image.jpg>', 'output': '#Import necessary libraries\nimport cv2\nimport numpy as np\n\n#Load the classifier for frontal face detection\nhaar_face_cascade = cv2.CascadeClassifier("haarcascade_frontal_face.xml")\n\n#Create a VideoCapture object\ncapture = cv2.VideoCapture("Sample_image.jpg")\nimg = capture.read()\n\n#Convert the image into grayscale\ngray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n\n#Detect facial features\nfaces = haar_face_cascade.detectMultiScale(gray, 1.3, 5)\n\n#Draw a rectangle around the faces\nfor (x,y,w,h) in faces:\n    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)\n\n#Write the image into a file\ncv2.imwrite(\'face_detect.jpg\',img)', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a Python script for recognizing facial fe

In [ ]:
# Check the dataset structure
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 1861
})


We would apply inference on those examples longer than 500 characters:

In [ ]:
# Sample the dataset
#sample_dataset= dataset[randrange(0,100,5)]
# Filter the dataset to get those which instruction is longer than 100 characters
responses_test = dataset.filter(lambda x: len(x['instruction'])>500)

print(len(responses_test))

Filter:   0%|          | 0/1861 [00:00<?, ? examples/s]

12


Now, we run this simple evaluation process

In [ ]:
# Process the test examples
times = []
solutions = []

for example in responses_test:
  start = time.time()
  prompt = f"""### Instruction:
  Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

  ### Task:
  {example["instruction"]}

  ### Input:
  {example["input"]}

  ### Response:
  """
  # Tokenize the input
  input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  # Run the model to infere an output
  outputs = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.3)

  # Collect the output and the inference time
  end = time.time()
  print(start-end)
  times.append(start-end)
  solutions.append(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):])

print(times)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


-9.108977317810059
-7.926240921020508
-7.896298408508301
-6.436765909194946
-7.823150396347046
-8.08249545097351
-8.073349237442017
-8.109416961669922
-8.120927333831787
-8.119935035705566
-6.791100978851318
-8.088157653808594
[-9.108977317810059, -7.926240921020508, -7.896298408508301, -6.436765909194946, -7.823150396347046, -8.08249545097351, -8.073349237442017, -8.109416961669922, -8.120927333831787, -8.119935035705566, -6.791100978851318, -8.088157653808594]


Show the metrics

In [ ]:
import numpy as np

In [ ]:
print('Min time: ',np.min(times))
print('Max time: ',np.max(times))
print('Mean time: ',np.mean(times))

Min time:  -9.108977317810059
Max time:  -6.436765909194946
Mean time:  -7.881401300430298
